<h1>IBM Data Science Capstone</h1>
<h3>Real Estate Investment Study Marseille Area</h3>

<img src="https://img.static-af.com/images/meta/IDname/CITY-MRS-1?aspect_ratio=12:5&max_width=1920" alt="Marseille" width="700"/>

In [1]:
!conda update -n base -c defaults conda --yes
!conda install -c conda-forge geopy --yes
!conda install folium -c conda-forge --yes
!conda install -c anaconda xlrd --yes
print('Packages installed.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tqdm-4.42.1                |             py_0          56 KB
    openssl-1.1.1d             |       h7b6447c_4         3.7 MB
    conda-package-handling-1.6.0|   py37h7b6447c_0         872 KB
    conda-4.8.2                |           py37_0         3.0 MB
    ------------------------------------------------------------
                                           Total:         7.7 MB

The following NEW packages will be INSTALLED:

    conda-package-handling: 1.6.0-py37h7b6447c_0
    tqdm:                   4.42.1-py_0         

The following packages will be UPDATED:

    conda:                  4.5.12-py37_0        --> 4.8.2-py37_0     
    openssl:                1.1.1d-h7b6447c_3    --> 1.1.1d-h7b6447c_4


tqdm-4.42.

In [2]:
import numpy as np
import pandas as pd
import requests # library to handle requests

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans

#Matplotlob
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

print('Libraries imported.')

Libraries imported.


In [3]:
#Foursquare Credentials
CLIENT_ID = '14SHI4R2ROE32BJMMJD3GLX2WDQT4SGSDYD3BDEI1QNJZP02' # your Foursquare ID
CLIENT_SECRET = 'SMEDPZUS4VKSLIBH1YWBISDO2424JWUISOF1CIHKSDGO1OLZ' # your Foursquare Secret
VERSION = '20200217'
YOUR_REGISTERED_REDIRECT_URI = 'https://www.google.com'
ACCESS_TOKEN = '21CDDM1QDGX5QFDIVIIBAN0E15FLJTK040I1QCD4H4SN4WJZ'
user_id = '484542633'
LIMIT = 30
print('Connect Foursquare API:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Connect Foursquare API:
CLIENT_ID: 14SHI4R2ROE32BJMMJD3GLX2WDQT4SGSDYD3BDEI1QNJZP02
CLIENT_SECRET:SMEDPZUS4VKSLIBH1YWBISDO2424JWUISOF1CIHKSDGO1OLZ


In [400]:
#Test adresse marseille
address = 'Marseille, France'
radius = 350
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.2961743 5.3699525


In [5]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=14SHI4R2ROE32BJMMJD3GLX2WDQT4SGSDYD3BDEI1QNJZP02&client_secret=SMEDPZUS4VKSLIBH1YWBISDO2424JWUISOF1CIHKSDGO1OLZ&ll=43.3029799,5.4299055&v=20200217&radius=1500&limit=30'

In [6]:
#Results of the request
results = requests.get(url).json()

#Get relevant part of JSON
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4c800a5e51ada1cd471e0610',
  'name': 'Le Terminus',
  'location': {'address': '2 place Caire',
   'lat': 43.303694115217674,
   'lng': 5.41883127107172,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.303694115217674,
     'lng': 5.41883127107172}],
   'distance': 900,
   'postalCode': '13012',
   'cc': 'FR',
   'city': 'Marseille',
   'state': "Provence-Alpes-Côte d'Azur",
   'country': 'France',
   'formattedAddress': ['2 place Caire', '13012 Marseille', 'France']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4c800a5e51ada1cd471e0610-0'}

<h4>We will now create a DataFrame with relevant data from json results</h4>

In [7]:
df_1 = json_normalize(items)
df_1.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c800a5e51ada1cd471e0610-0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",4c800a5e51ada1cd471e0610,2 place Caire,FR,Marseille,France,NaN,...,"[2 place Caire, 13012 Marseille, France]","[{'label': 'display', 'lat': 43.30369411521767...",43.303694,5.418831,13012,Provence-Alpes-Côte d'Azur,Le Terminus,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b5ec1b1f964a520bc9729e3-1,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",4b5ec1b1f964a520bc9729e3,NaN,FR,Marseille,France,NaN,...,"[13012 Marseille, France]","[{'label': 'display', 'lat': 43.30430556726937...",43.304306,5.415143,13012,Provence-Alpes-Côte d'Azur,Saint-Barnabé,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4f76da22e4b015b23c73f901-2,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",4f76da22e4b015b23c73f901,37 rue Gaston De Flotte,FR,Marseille,France,St Jean Du Desert,...,"[37 rue Gaston De Flotte (St Jean Du Desert), ...","[{'label': 'display', 'lat': 43.296051, 'lng':...",43.296051,5.425035,13012,Provence-Alpes-Côte d'Azur,Intermarché SUPER,0,[],577316391
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5421792c498edfcb92e8cd2b-3,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",5421792c498edfcb92e8cd2b,rue Léon Meisserel,FR,Marseille,France,NaN,...,"[rue Léon Meisserel, 13012 Marseille, France]","[{'label': 'display', 'lat': 43.304861, 'lng':...",43.304861,5.418489,13012,Provence-Alpes-Côte d'Azur,Market,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b913f12f964a520e0ac33e3-4,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",4b913f12f964a520e0ac33e3,693 avenue De Mazargues,FR,Marseille,France,Centre Commercial Saint Barnabé Village,...,[693 avenue De Mazargues (Centre Commercial Sa...,"[{'label': 'display', 'lat': 43.30473999157467...",43.304740,5.418042,13009,AUVERGNE,Carrefour City,0,[],NaN


In [8]:
#Define information of interest and filter dataframe

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h4>Venues located near 1er arrondissement Borough</h4>

In [9]:
#Process JSON and convert it to a clean dataframe
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Le Terminus,Café,2 place Caire,FR,Marseille,France,NaN,900,"[2 place Caire, 13012 Marseille, France]","[{'label': 'display', 'lat': 43.30369411521767...",43.303694,5.418831,13012,Provence-Alpes-Côte d'Azur,4c800a5e51ada1cd471e0610
1,Saint-Barnabé,Neighborhood,NaN,FR,Marseille,France,NaN,1204,"[13012 Marseille, France]","[{'label': 'display', 'lat': 43.30430556726937...",43.304306,5.415143,13012,Provence-Alpes-Côte d'Azur,4b5ec1b1f964a520bc9729e3
2,Intermarché SUPER,Supermarket,37 rue Gaston De Flotte,FR,Marseille,France,St Jean Du Desert,866,"[37 rue Gaston De Flotte (St Jean Du Desert), ...","[{'label': 'display', 'lat': 43.296051, 'lng':...",43.296051,5.425035,13012,Provence-Alpes-Côte d'Azur,4f76da22e4b015b23c73f901
3,Market,Supermarket,rue Léon Meisserel,FR,Marseille,France,NaN,948,"[rue Léon Meisserel, 13012 Marseille, France]","[{'label': 'display', 'lat': 43.304861, 'lng':...",43.304861,5.418489,13012,Provence-Alpes-Côte d'Azur,5421792c498edfcb92e8cd2b
4,Carrefour City,Supermarket,693 avenue De Mazargues,FR,Marseille,France,Centre Commercial Saint Barnabé Village,980,[693 avenue De Mazargues (Centre Commercial Sa...,"[{'label': 'display', 'lat': 43.30473999157467...",43.304740,5.418042,13009,AUVERGNE,4b913f12f964a520e0ac33e3
5,Simply market,Grocery Store,NaN,FR,NaN,France,NaN,296,[France],"[{'label': 'display', 'lat': 43.30470712407533...",43.304707,5.427112,NaN,NaN,4b8fdf90f964a520b36633e3
6,L'entrecôte Tao,French Restaurant,NaN,FR,NaN,France,NaN,720,[France],"[{'label': 'display', 'lat': 43.307649, 'lng':...",43.307649,5.423746,NaN,NaN,4c8cc56ccf3ea1432c0df851
7,BP,Gas Station,53 Avenue de Saint-Barnabé,FR,Marseille,France,NaN,1491,"[53 Avenue de Saint-Barnabé, 13012 Marseille, ...","[{'label': 'display', 'lat': 43.304322, 'lng':...",43.304322,5.411587,13012,Provence-Alpes-Côte d'Azur,550b38c8498efa0b2adeef8a
8,Station La Grognarde [T1],Tram Station,31 Traverse des Faienciers,FR,Marseille,France,NaN,950,"[31 Traverse des Faienciers, 13012 Marseille, ...","[{'label': 'display', 'lat': 43.29461763587872...",43.294618,5.432283,13012,PACA,4dfe0a54091aaaba729ece8c
9,Station Air Bel [T1],Tram Station,2 Traverse des Faienciers,FR,Marseille,France,NaN,980,"[2 Traverse des Faienciers, 13012 Marseille, F...","[{'label': 'display', 'lat': 43.29431299982423...",43.294313,5.427752,13012,PACA,4dfe09c9091aaaba729ece35


<h2>Trending places ?</h2>

In [53]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e63ee243907e700280ffadd'},
 'response': {'venues': []}}

In [54]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [55]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

<h1>Chloropleth Map</h1>

In [13]:
import folium

In [402]:
# create a plain  map
marseille_map = folium.Map(location=[43.300672, 5.381938], zoom_start=12)
marseille_map

<h3>Prices of Marseilles Real Estate are extracted from the following website and put in an .xlsx file. <br /> We will get this date and merge it with Foursquare location data later. <br />In the meantime, you will find below a view of prices / borough on a map</h3>

In [15]:
#import price / neighborhood table
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_42f19aa0e23e440daaa33eb80ced6d5a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='qAlLaWx17nVZQyTZwU4MvWT3wYqr_Wf8YqcP_N6hskf7',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_42f19aa0e23e440daaa33eb80ced6d5a.get_object(Bucket='advancedatasciencecapstone-donotdelete-pr-vcj0xk3stcpg1k',Key='prix immo marseille.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_a = pd.read_excel(body, skiprows=range(1))
df_a.head()

,DEPCO,Borough,Average_sale_price_house(€_m2),Average_sale_price_appt(€_m2)
0,13201,Marseille 1er Arrondissemen,4099,2530
1,13202,Marseille 2e Arrondissemen,3567,2972
2,13203,Marseille 3e Arrondissemen,1868,1737
3,13204,Marseille 4e Arrondissemen,3027,2249
4,13205,Marseille 5e Arrondissemen,3150,2671


In [16]:
df_a.rename(columns={"Average_sale_price_house(€_m2)": "Price_house_m2", "Average_sale_price_appt(€_m2)": "Price_appt_m2"}, inplace=True)
df_a.head()

,DEPCO,Borough,Price_house_m2,Price_appt_m2
0,13201,Marseille 1er Arrondissemen,4099,2530
1,13202,Marseille 2e Arrondissemen,3567,2972
2,13203,Marseille 3e Arrondissemen,1868,1737
3,13204,Marseille 4e Arrondissemen,3027,2249
4,13205,Marseille 5e Arrondissemen,3150,2671


<h4>I need to convert the Postal Code field into Str in order to have it recognized as the key for the geojson file</h4>

In [17]:
df_a[["DEPCO"]] = df_a[["DEPCO"]].astype(str)
df_a.dtypes

DEPCO             object
Borough           object
Price_house_m2     int64
Price_appt_m2      int64
dtype: object

<h3>Average price of appartments (€/m2) Marseille's Borough</h3>

In [403]:
mrs_geo = 'https://public.opendatasoft.com/explore/dataset/arrondissements-millesimes0/download/?format=geojson&refine.code_dpartement=13&timezone=Europe/Berlin&lang=fr'
mrs_data = df_a[['DEPCO','Borough','Price_appt_m2']]

map_appts = folium.Map(location=[43.300672, 5.381938], zoom_start=11, tiles='cartodbpositron')

folium.Choropleth(
    geo_data=mrs_geo,
    name='choropleth',
    data=mrs_data,
    columns=['DEPCO','Price_appt_m2'],
    key_on='feature.properties.code_insee',
    fill_color='YlOrBr',
    fill_opacity=0.5,
    line_opacity=0.7,
    legend_name='Price appartments Marseille m2'
).add_to(map_appts)

folium.LayerControl().add_to(map_appts)

map_appts

<h3>Average price of houses (€/m2) Marseille's Borough</h3>

In [19]:
mrs_geo = 'https://public.opendatasoft.com/explore/dataset/arrondissements-millesimes0/download/?format=geojson&refine.code_dpartement=13&timezone=Europe/Berlin&lang=fr'
mrs_data = df_a[['DEPCO','Borough','Price_house_m2']]

map_houses = folium.Map(location=[43.300672, 5.381938], zoom_start=11, tiles='cartodbpositron')

folium.Choropleth(
    geo_data=mrs_geo,
    name='choropleth',
    data=mrs_data,
    columns=['DEPCO','Price_house_m2'],
    key_on='feature.properties.code_insee',
    fill_color='YlOrBr',
    fill_opacity=0.5,
    line_opacity=0.7,
    legend_name='Price houses Marseille m2'
).add_to(map_houses)

folium.LayerControl().add_to(map_houses)

map_houses

I want to study the differences between the **10 most expensive and 10 less expensive neighborhoods of Marseille** based on their Foursquare places differences.<br /><br />
To start we can observe that there is a big difference between appartment prices (less expensive in the North of the city, 14th and 15th Borough) while on the other hand, houses prices in the south close to the sea are 4x higher and way more expensive than the rest of the city.<br /><br />
Now that we have the Borough, I want to know by their respective Neighborhoods. <br />
That will also give us the address to look for with Foursquare locator and get latitude / longitude

In [20]:
body = client_42f19aa0e23e440daaa33eb80ced6d5a.get_object(Bucket='advancedatasciencecapstone-donotdelete-pr-vcj0xk3stcpg1k',Key='test_cholopleth.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_n = pd.read_excel(body, skiprows=range(1))
df_n.head()

,Postal_Code,Borough,Neighborhood,Average_sale_price_house(€_m2),Average_sale_price_appt(€_m2),DEPCO,NOM_CO
0,13001,1er Arrondissement,OPERA,3564,3216,13201,Marseille 1er Arrondissemen
1,13001,1er Arrondissement,BELSUNCE,4350,2582,13201,Marseille 1er Arrondissemen
2,13001,1er Arrondissement,SAINT CHARLES,4094,2696,13201,Marseille 1er Arrondissemen
3,13001,1er Arrondissement,NOAILLES,4084,2292,13201,Marseille 1er Arrondissemen
4,13001,1er Arrondissement,THIERS,4099,2560,13201,Marseille 1er Arrondissemen


In [21]:
df_n.rename(columns={"Average_sale_price_house(€_m2)": "Price_house_m2", "Average_sale_price_appt(€_m2)": "Price_appt_m2"}, inplace=True)
df_n.drop(columns=['NOM_CO'])
df_n.head()

,Postal_Code,Borough,Neighborhood,Price_house_m2,Price_appt_m2,DEPCO
0,13001,1er Arrondissement,OPERA,3564,3216,13201
1,13001,1er Arrondissement,BELSUNCE,4350,2582,13201
2,13001,1er Arrondissement,SAINT CHARLES,4094,2696,13201
3,13001,1er Arrondissement,NOAILLES,4084,2292,13201
4,13001,1er Arrondissement,THIERS,4099,2560,13201
5,13001,1er Arrondissement,CHAPITRE,4087,2437,13201
6,13002,2e Arrondissement,HOTEL DE VILLE,4193,3374,13202
7,13002,2e Arrondissement,LA JOLIETTE,3576,2882,13202
8,13002,2e Arrondissement,GRANDS CARMES,3577,2825,13202
9,13002,2e Arrondissement,ARENC,3296,2702,13202


In [22]:
df_sorted = df_n.sort_values(by=['Price_house_m2'], ascending=False)
df_richest = df_sorted.head(10)
df_richest = df_richest.reset_index(drop=True)
df_richest

,Postal_Code,Borough,Neighborhood,Price_house_m2,Price_appt_m2,DEPCO,NOM_CO
0,13007,7e Arrondissement,ROUCAS BLANC,6913,5388,13207,Marseille 7e Arrondissemen
1,13007,7e Arrondissement,BOMPARD,6754,4774,13207,Marseille 7e Arrondissemen
2,13007,7e Arrondissement,LE PHARO,6213,3812,13207,Marseille 7e Arrondissemen
3,13007,7e Arrondissement,SAINT VICTOR,6204,3677,13207,Marseille 7e Arrondissemen
4,13007,7e Arrondissement,ENDOUME,5753,4171,13207,Marseille 7e Arrondissemen
5,13008,8e Arrondissement,LA PLAGE,5608,5565,13208,Marseille 8e Arrondissemen
6,13008,8e Arrondissement,SAINT GINIEZ,5566,3627,13208,Marseille 8e Arrondissemen
7,13008,8e Arrondissement,MONTREDON,5189,4194,13208,Marseille 8e Arrondissemen
8,13008,8e Arrondissement,POINTE ROUGE,5103,3173,13208,Marseille 8e Arrondissemen
9,13008,8e Arrondissement,PERIER,5057,3550,13208,Marseille 8e Arrondissemen


In [23]:
df_sorted_low = df_n.sort_values(by=['Price_appt_m2'])
df_poorest = df_sorted_low.head(10)
df_poorest = df_poorest.reset_index(drop=True)
df_poorest

,Postal_Code,Borough,Neighborhood,Price_house_m2,Price_appt_m2,DEPCO,NOM_CO
0,13015,15e Arrondissement,NOTRE DAME LIMITE,2555,1156,13215,Marseille 15e Arrondissemen
1,13015,15e Arrondissement,LA CABUCELLE,1876,1231,13215,Marseille 15e Arrondissemen
2,13015,15e Arrondissement,LA DELORME,2556,1234,13215,Marseille 15e Arrondissemen
3,13015,15e Arrondissement,SAINT ANTOINE,2380,1257,13215,Marseille 15e Arrondissemen
4,13015,15e Arrondissement,LA CALADE,2456,1373,13215,Marseille 15e Arrondissemen
5,13014,14e Arrondissement,SAINT JOSEPH,2316,1416,13214,Marseille 14e Arrondissemen
6,13014,14e Arrondissement,LES ARNAVAUX,2397,1451,13214,Marseille 14e Arrondissemen
7,13014,14e Arrondissement,LE CANET,1912,1474,13214,Marseille 14e Arrondissemen
8,13015,15e Arrondissement,VERDURON,2824,1513,13215,Marseille 15e Arrondissemen
9,13015,15e Arrondissement,LES CROTTES,2458,1515,13215,Marseille 15e Arrondissemen


<h2>Get Latitude Longitude of TOP10 & LOW10 Marseille's neighborhoods</h2>

In [24]:
df_agg = pd.concat([df_richest, df_poorest])
df_agg.drop(columns=['NOM_CO'], inplace=True)

In [25]:
df_agg.reset_index(drop=True, inplace=True)
df_agg.head()

,Postal_Code,Borough,Neighborhood,Price_house_m2,Price_appt_m2,DEPCO
0,13007,7e Arrondissement,ROUCAS BLANC,6913,5388,13207
1,13007,7e Arrondissement,BOMPARD,6754,4774,13207
2,13007,7e Arrondissement,LE PHARO,6213,3812,13207
3,13007,7e Arrondissement,SAINT VICTOR,6204,3677,13207
4,13007,7e Arrondissement,ENDOUME,5753,4171,13207


In [31]:
# define the dataframe columns
column_names = ['N_Address', 'Latitude', 'Longitude'] 

# initiate the dataframe
df_n20 = pd.DataFrame(columns=column_names)
df_n20

,N_Address,Latitude,Longitude


In [32]:
geolocator = Nominatim(user_agent="foursquare_agent")
for i in range (0,20):
    address = "{}, {} Marseille, France".format(df_agg.iloc[i,2], df_agg.iloc[i,0])
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    df_n20 = df_n20.append({'N_Address': "{}, {} Marseille, France".format(df_agg.iloc[i,2], df_agg.iloc[i,0]),
                                                'Latitude': latitude,
                                                'Longitude': longitude}, ignore_index=True)
df_n20

,N_Address,Latitude,Longitude
0,"ROUCAS BLANC, 13007 Marseille, France",43.276499,5.365867
1,"BOMPARD, 13007 Marseille, France",43.280460,5.360713
2,"LE PHARO, 13007 Marseille, France",43.291300,5.359486
3,"SAINT VICTOR, 13007 Marseille, France",43.289355,5.366286
4,"ENDOUME, 13007 Marseille, France",43.282862,5.352171
5,"LA PLAGE, 13008 Marseille, France",43.262602,5.375698
6,"SAINT GINIEZ, 13008 Marseille, France",43.268089,5.388772
7,"MONTREDON, 13008 Marseille, France",43.228695,5.364547
8,"POINTE ROUGE, 13008 Marseille, France",43.243715,5.370129
9,"PERIER, 13008 Marseille, France",43.274175,5.378945


Here are the locations of each neighborhood on a map. <br />Blue the most expensive. Red the least

In [33]:
# Marseilles map 
map_mrs3 = folium.Map(location=[43.300672, 5.381938], zoom_start=12)
df1 = df_n20.head(10)
df2 = df_n20.tail(10)

# add markers to map
for lat, lng, label in zip(df1['Latitude'], df1['Longitude'], df1['N_Address']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='navy',
        fill=True,
        fill_color='lightblue',
        fill_opacity=0.3,
        parse_html=False).add_to(map_mrs3)  
    
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['N_Address']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='orangered',
        fill=True,
        fill_color='coral',
        fill_opacity=0.3,
        parse_html=False).add_to(map_mrs3)  

map_mrs3

Now we will study what these neighborhoods are made of with Foursquare data

In [316]:
#Here is the function to repeat "find nearby venues" with Foursquare for each address
def getNearbyVenues(names, latitudes, longitudes, radius=300, LIMIT=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
df_add = df_agg["Neighborhood"]

In [36]:
df_n20 = pd.concat([df_n20, df_add], axis=1)
df_n20.head()

,N_Address,Latitude,Longitude,Neighborhood
0,"ROUCAS BLANC, 13007 Marseille, France",43.276499,5.365867,ROUCAS BLANC
1,"BOMPARD, 13007 Marseille, France",43.280460,5.360713,BOMPARD
2,"LE PHARO, 13007 Marseille, France",43.291300,5.359486,LE PHARO
3,"SAINT VICTOR, 13007 Marseille, France",43.289355,5.366286,SAINT VICTOR
4,"ENDOUME, 13007 Marseille, France",43.282862,5.352171,ENDOUME


In [37]:
#split again
df_top10 = df_n20.head(10)
df_low10 = df_n20.tail(10)

In [38]:
mrs_venues1 = getNearbyVenues(names=df_top10['Neighborhood'],
                                   latitudes=df_top10['Latitude'],
                                   longitudes=df_top10['Longitude']
                                  )
mrs_venues1.head()

ROUCAS BLANC
BOMPARD
LE PHARO
SAINT VICTOR
ENDOUME
LA PLAGE
SAINT GINIEZ
MONTREDON
POINTE ROUGE
PERIER


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ROUCAS BLANC,43.276499,5.365867,Plage du Prophète,43.273143,5.362101,Beach
1,ROUCAS BLANC,43.276499,5.365867,Place du Terrail,43.279751,5.368206,Plaza
2,ROUCAS BLANC,43.276499,5.365867,La Reserve,43.274675,5.360796,French Restaurant
3,ROUCAS BLANC,43.276499,5.365867,"Arrêt Le Prophète [83,583]",43.272995,5.362621,Bus Stop
4,ROUCAS BLANC,43.276499,5.365867,Serafina,43.275573,5.359984,Café


In [39]:
mrs_venues1.shape

(118, 7)

In [40]:
mrs_venues1.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BOMPARD,6,6,6,6,6,6
ENDOUME,21,21,21,21,21,21
LA PLAGE,11,11,11,11,11,11
LE PHARO,26,26,26,26,26,26
PERIER,2,2,2,2,2,2
POINTE ROUGE,12,12,12,12,12,12
ROUCAS BLANC,6,6,6,6,6,6
SAINT GINIEZ,11,11,11,11,11,11
SAINT VICTOR,23,23,23,23,23,23


In [41]:
# one hot encoding
mrs_onehot1 = pd.get_dummies(mrs_venues1[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mrs_onehot1['Neighborhood'] = mrs_venues1['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mrs_onehot1.columns[-1]] + list(mrs_onehot1.columns[:-1])
mrs_onehot1 = mrs_onehot1[fixed_columns]

mrs_onehot1.head()

,Neighborhood,Bakery,Bar,Bay,Beach,Boat or Ferry,Burger Joint,Bus Stop,Café,Chinese Restaurant,...,Seafood Restaurant,Shopping Mall,Speakeasy,Sporting Goods Shop,Stadium,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Waterfront
0,ROUCAS BLANC,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ROUCAS BLANC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ROUCAS BLANC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ROUCAS BLANC,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ROUCAS BLANC,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
mrs_onehot1 = mrs_onehot1.groupby('Neighborhood').mean().reset_index()
mrs_onehot1

,Neighborhood,Bakery,Bar,Bay,Beach,Boat or Ferry,Burger Joint,Bus Stop,Café,Chinese Restaurant,...,Seafood Restaurant,Shopping Mall,Speakeasy,Sporting Goods Shop,Stadium,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Waterfront
0,BOMPARD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.166667
1,ENDOUME,0.047619,0.000000,0.095238,0.047619,0.000000,0.000000,0.000000,0.000000,0.047619,...,0.095238,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000
2,LA PLAGE,0.000000,0.000000,0.000000,0.272727,0.000000,0.000000,0.090909,0.000000,0.000000,...,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,LE PHARO,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.076923,0.000000,...,0.038462,0.000000,0.000000,0.000000,0.038462,0.038462,0.038462,0.000000,0.000000,0.000000
4,PERIER,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,POINTE ROUGE,0.000000,0.000000,0.083333,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000
6,ROUCAS BLANC,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.333333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,SAINT GINIEZ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000
8,SAINT VICTOR,0.086957,0.043478,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,...,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000


In [43]:
num_top_venues = 5

for hood in mrs_onehot1['Neighborhood']:
    print("----"+hood+"----")
    temp = mrs_onehot1[mrs_onehot1['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BOMPARD----
           venue  freq
0     Waterfront  0.17
1          Plaza  0.17
2        Theater  0.17
3  Grocery Store  0.17
4    Gas Station  0.17


----ENDOUME----
                  venue  freq
0           Pizza Place   0.1
1                   Bay   0.1
2                 Plaza   0.1
3  Provençal Restaurant   0.1
4    Seafood Restaurant   0.1


----LA PLAGE----
                      venue  freq
0                     Beach  0.27
1         Outdoor Sculpture  0.18
2                     Hotel  0.09
3  Mediterranean Restaurant  0.09
4                  Bus Stop  0.09


----LE PHARO----
             venue  freq
0            Hotel  0.12
1           Lounge  0.08
2    Historic Site  0.08
3  Harbor / Marina  0.08
4             Café  0.08


----PERIER----
        venue  freq
0       Diner   0.5
1  Restaurant   0.5
2      Bakery   0.0
3        Pool   0.0
4      Lounge   0.0


----POINTE ROUGE----
                      venue  freq
0  Mediterranean Restaurant  0.17
1                Restaurant 

As a conclusion for these top 10 neighborhoods of Marseille, they have in common places like : Waterfront - Harbor - Beach - Marina, many Hotels, Historic site, French - Italian - Mediteranean restaurants. <br /> In comparison, let's do the same with the 10 least attractive suburbs.

In [44]:
mrs_venues2 = getNearbyVenues(names=df_low10['Neighborhood'],
                                   latitudes=df_low10['Latitude'],
                                   longitudes=df_low10['Longitude']
                                  )
mrs_venues2.head()

NOTRE DAME LIMITE
LA CABUCELLE
LA DELORME
SAINT ANTOINE
LA CALADE
SAINT JOSEPH
LES ARNAVAUX
LE CANET
VERDURON
LES CROTTES


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,NOTRE DAME LIMITE,43.381963,5.365024,"Arrêt Hôpital Nord Urgences [B2,96,97]",43.379413,5.361435,Bus Stop
1,LA CABUCELLE,43.337239,5.357562,Le Café des Halles de la Major,43.335526,5.360526,Diner
2,LA CABUCELLE,43.337239,5.357562,Roig More,43.341116,5.357173,Furniture / Home Store
3,LA CABUCELLE,43.337239,5.357562,"Arrêt Lyon Raffineries [B2,25]",43.336002,5.363195,Bus Stop
4,LA DELORME,43.346146,5.371247,restaurant el carino,43.348703,5.373462,French Restaurant


In [45]:
mrs_venues2.shape

(27, 7)

<h3>In comparison, Top 10 had >120 venues nearby</h3>

In [46]:
mrs_venues2.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
LA CABUCELLE,3,3,3,3,3,3
LA CALADE,2,2,2,2,2,2
LA DELORME,2,2,2,2,2,2
LE CANET,3,3,3,3,3,3
LES ARNAVAUX,3,3,3,3,3,3
LES CROTTES,4,4,4,4,4,4
NOTRE DAME LIMITE,1,1,1,1,1,1
SAINT ANTOINE,5,5,5,5,5,5
SAINT JOSEPH,4,4,4,4,4,4


<h3>We will now compute the frequency of each categoy per Neighborhood, essential step before clustering, as K-Means only takes numbers into account</h3>

In [47]:
# one hot encoding
mrs_onehot2 = pd.get_dummies(mrs_venues2[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mrs_onehot2['Neighborhood'] = mrs_venues2['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mrs_onehot2.columns[-1]] + list(mrs_onehot2.columns[:-1])
mrs_onehot2 = mrs_onehot2[fixed_columns]

mrs_onehot2.head()

,Neighborhood,Bus Stop,Candy Store,Clothing Store,Construction & Landscaping,Diner,Discount Store,Fast Food Restaurant,French Restaurant,Furniture / Home Store,...,Metro Station,Miscellaneous Shop,Photography Studio,Platform,Rental Car Location,Rental Service,Restaurant,Snack Place,Stadium,Train Station
0,NOTRE DAME LIMITE,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,LA CABUCELLE,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,LA CABUCELLE,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,LA CABUCELLE,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LA DELORME,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
mrs_onehot2 = mrs_onehot2.groupby('Neighborhood').mean().reset_index()

<h3>Here are the 3 most frequent venues per Neighborhood</h3>

In [49]:
num_top_venues = 3

for hood in mrs_onehot2['Neighborhood']:
    print("----"+hood+"----")
    temp = mrs_onehot2[mrs_onehot2['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----LA CABUCELLE----
                    venue  freq
0                Bus Stop  0.33
1                   Diner  0.33
2  Furniture / Home Store  0.33


----LA CALADE----
                    venue  freq
0          Rental Service   0.5
1  Furniture / Home Store   0.5
2                Bus Stop   0.0


----LA DELORME----
               venue  freq
0  French Restaurant   1.0
1           Bus Stop   0.0
2      Metro Station   0.0


----LE CANET----
        venue  freq
0       Diner  0.33
1  Restaurant  0.33
2    Platform  0.33


----LES ARNAVAUX----
                 venue  freq
0          Candy Store  0.33
1  Rental Car Location  0.33
2   Miscellaneous Shop  0.33


----LES CROTTES----
                        venue  freq
0                       Hotel  0.25
1               Metro Station  0.25
2  Construction & Landscaping  0.25


----NOTRE DAME LIMITE----
         venue  freq
0     Bus Stop   1.0
1  Candy Store   0.0
2      Stadium   0.0


----SAINT ANTOINE----
            venue  freq
0   Train 

For half of these suburbs, there is only one venue <br />
We can see for the other that bus stop, train station or gas station are not really attractive.<br />
The luckiest have stores or boutiques nearby

<h3>Clustering</h3>

Now the goal as an investor is to find places similar to the most expensive, but in other Borough. <br />
Of course, socio-economic data, such as education, income etc are relevant to study this situation. <br />
However as this lab is focused on Foursquare Data, let's cluster the data we have and observe oif we can find something interesting.

To start, we will do what we did for the top/low10 for the whole city now

In [270]:
df_n.shape

(110, 7)

In [271]:
df_n.tail()

,Postal_Code,Borough,Neighborhood,Price_house_m2,Price_appt_m2,DEPCO,NOM_CO
105,13015,15e Arrondissement,LES CROTTES,2458,1515,13215,Marseille 15e Arrondissemen
106,13016,16e Arrondissement,L'ESTAQUE,3248,2691,13216,Marseille 16e Arrondissemen
107,13016,16e Arrondissement,SAINT HENRI,3038,3119,13216,Marseille 16e Arrondissemen
108,13016,16e Arrondissement,LES RIAUX,2948,2666,13216,Marseille 16e Arrondissemen
109,13016,16e Arrondissement,SAINT ANDRE,2242,2548,13216,Marseille 16e Arrondissemen


In [289]:
#There is a spelling mistake line 12,37 and 89 that we need to correct
df_n.replace({'VIELLE CHAPELLE' : 'VIEILLE CHAPPELLE'},{'SAINT MARTHE' : 'SAINTE MARTHE'},{'SAINT MAURON' : 'SAINT-MAURONT'}, inplace=True)

In [273]:
#Add latitude longitude for each neighborhood in the city
# define the dataframe columns
columns_names = ['N_Address', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_city = pd.DataFrame(columns=columns_names)
df_city

,N_Address,Latitude,Longitude


In [300]:
for i in range (103,110):
    address = "{}, {} Marseille, France".format(df_n.iloc[i,2], df_n.iloc[i,0])
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    df_city = df_city.append({'N_Address': "{}, {} Marseille, France".format(df_n.iloc[i,2], df_n.iloc[i,0]),
                                                'Latitude': latitude,
                                                'Longitude': longitude}, ignore_index=True)
df_city.tail(5)

,N_Address,Latitude,Longitude
105,"LES CROTTES, 13015 Marseille, France",43.324116,5.367809
106,"L'ESTAQUE, 13016 Marseille, France",43.364509,5.316188
107,"SAINT HENRI, 13016 Marseille, France",43.360192,5.331872
108,"LES RIAUX, 13016 Marseille, France",43.363126,5.304214
109,"SAINT ANDRE, 13016 Marseille, France",43.352387,5.342880


In [317]:
df_city.shape

(110, 4)

In [ ]:
#add colmun neighborhood (only)
df_add = df_n["Neighborhood"]
df_city = pd.concat([df_city, df_add], axis=1)

In [335]:
#get nearby venues by latitude longitude, cf fonction created before
city_venues = getNearbyVenues(names=df_city['Neighborhood'],
                                   latitudes=df_city['Latitude'],
                                   longitudes=df_city['Longitude']
                                  )
city_venues.shape

OPERA
BELSUNCE
SAINT CHARLES
NOAILLES
THIERS
CHAPITRE
HOTEL DE VILLE
LA JOLIETTE
GRANDS CARMES
ARENC
LA VILLETTE
SAINT LAZARE
SAINT-MAURONT
BELLE DE MAI
CINQ AVENUES
LA BLANCARDE
LES CHARTREUX
CHUTES LAVIE
BAILLE
LE CAMAS
LA CONCEPTION
SAINT PIERRE
VAUBAN
PALAIS DE JUSTICE
CASTELLANE
PREFECTURE
LODI
NOTRE DAME DU MONT
ROUCAS BLANC
BOMPARD
ENDOUME
SAINT LAMBERT
LE PHARO
SAINT VICTOR
LA PLAGE
LES GOUDES
MONTREDON
VIEILLE CHAPELLE
PERIER
SAINT GINIEZ
BONNEVEINE
SAINTE ANNE
LE ROUET
POINTE ROUGE
VAUFREGES
SORMIOU
LES BAUMETTES
MAZARGUES
LE CABOT
CARPIAGNE
LE REDON
SAINTE MARGUERITE
LA PANOUSE
LA TIMONE
LA CAPELETTE
MENPENTI
PONT DE VIVAUX
SAINT TRONC
SAINT LOUP
LA TREILLE
LES CAMOINS
LES ACCATES
LA VALENTINE
EOURES
SAINT MENET
LA MILLIERE
LA BARASSE
SAINT MARCEL
LA POMME
LA VALBARELLE
LES TROIS LUCS
SAINT BARNABE
SAINT JEAN DU DESERT
SAINT JULIEN
LES CAILLOLS
LA FOURRAGERE
MONTOLIVET
LES MEDECINS
CHATEAU-GOMBERT
PALAMA
LA CROIX ROUGE
SAINT MITRE
LES OLIVES
LES MOURETS
SAINT JEROME
SAINT JU

(555, 7)

In [337]:
city_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ARENC,1,1,1,1,1,1
BAILLE,4,4,4,4,4,4
BELLE DE MAI,3,3,3,3,3,3
BELSUNCE,16,16,16,16,16,16
BOMPARD,3,3,3,3,3,3
BON SECOURS,1,1,1,1,1,1
BONNEVEINE,2,2,2,2,2,2
CASTELLANE,18,18,18,18,18,18
CHAPITRE,15,15,15,15,15,15


In [334]:
city_venues.shape

(555, 7)

In [384]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Neighborhood'] = city_venues['Neighborhood'] 
#df_add = city_venues["Neighborhood"]
#city_onehot = pd.concat([city_onehot, df_add], axis=1)

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Neighborhood,Accessories Store,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Auto Workshop,Automotive Shop,Bagel Shop,Bakery,...,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant
0,OPERA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,OPERA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,OPERA,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,OPERA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,OPERA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [385]:
city_onehot = city_onehot.groupby('Neighborhood').mean().reset_index()
city_onehot.tail()

,Neighborhood,Accessories Store,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Auto Workshop,Automotive Shop,Bagel Shop,Bakery,...,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant
85,SORMIOU,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,THIERS,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,VAUBAN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,VAUFREGES,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,VIEILLE CHAPELLE,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [386]:
#Define function for top 5 most common venues into DataFrame
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [387]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = city_onehot['Neighborhood']

for ind in np.arange(city_onehot.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_onehot.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,ARENC,Pool,Vegetarian / Vegan Restaurant,Dance Studio,Event Service,Electronics Store,Diner,Dessert Shop
1,BAILLE,Supermarket,Bakery,Coffee Shop,Event Service,Electronics Store,Diner,Dessert Shop
2,BELLE DE MAI,Supermarket,Art Gallery,Theater,Cupcake Shop,Event Service,Electronics Store,Diner
3,BELSUNCE,Hotel,Bistro,Garden,Indie Movie Theater,Metro Station,Middle Eastern Restaurant,Electronics Store
4,BOMPARD,Bus Stop,Pizza Place,Grocery Store,Cupcake Shop,Electronics Store,Diner,Dessert Shop


<h3>Kmeans Machine Learning</h3>

In [388]:
city_onehot = city_onehot.join(df_n.set_index('Neighborhood'), on='Neighborhood')

In [389]:
city_onehot.drop(columns=['Borough','DEPCO','NOM_CO'], inplace=True)
city_onehot.head()

,Neighborhood,Accessories Store,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Auto Workshop,Automotive Shop,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Postal_Code,Price_house_m2,Price_appt_m2
0,ARENC,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,13002,3296,2702
1,BAILLE,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,13005,3385,2767
2,BELLE DE MAI,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.333333,0.0,0.0,0.0000,0.0,0.0,13003,2256,1681
3,BELSUNCE,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0625,0.0,0.0,13001,4350,2582
4,BOMPARD,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,13007,6754,4774


In [390]:
# set number of clusters
kclusters = 4

clustering = city_onehot.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, n_init = 12, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 0, 1, 2, 0, 1, 1, 1, 1], dtype=int32)

In [391]:
kmeans.labels_

array([3, 3, 0, 1, 2, 0, 1, 1, 1, 1, 3, 3, 2, 3, 3, 1, 3, 3, 0, 0, 3, 3,
       3, 0, 3, 3, 2, 3, 0, 3, 1, 3, 1, 0, 0, 1, 3, 0, 3, 2, 1, 1, 1, 0,
       0, 3, 1, 3, 3, 1, 1, 3, 1, 3, 3, 3, 1, 1, 3, 1, 3, 1, 2, 0, 0, 1,
       1, 2, 3, 1, 0, 1, 3, 1, 0, 3, 3, 1, 3, 3, 3, 2, 0, 1, 0, 1, 1, 1,
       1, 1], dtype=int32)

In [392]:
df_final = df_n.drop(columns=['NOM_CO','DEPCO'])
df_final.head()

,Postal_Code,Borough,Neighborhood,Price_house_m2,Price_appt_m2
0,13001,1er Arrondissement,OPERA,3564,3216
1,13001,1er Arrondissement,BELSUNCE,4350,2582
2,13001,1er Arrondissement,SAINT CHARLES,4094,2696
3,13001,1er Arrondissement,NOAILLES,4084,2292
4,13001,1er Arrondissement,THIERS,4099,2560


In [393]:
df_final = df_final.join(df_city.set_index('Neighborhood'), on='Neighborhood')
df_final.drop(columns=['N_Address'],inplace=True)
#If parameters changed, we need to empty Cluster Columns
#neighborhoods_venues_sorted.drop(columns=['Cluster Labels'],inplace=True)

In [394]:
# add clustering labels column in our dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge to add latitude/longitude for each neighborhood
df_final = df_final.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_final.head() # check the last columns!

,Postal_Code,Borough,Neighborhood,Price_house_m2,Price_appt_m2,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,13001,1er Arrondissement,OPERA,3564,3216,43.293307,5.375022,1.0,French Restaurant,Pub,Provençal Restaurant,Plaza,Boutique,Coffee Shop,Burger Joint
1,13001,1er Arrondissement,BELSUNCE,4350,2582,43.299124,5.377096,1.0,Hotel,Bistro,Garden,Indie Movie Theater,Metro Station,Middle Eastern Restaurant,Electronics Store
2,13001,1er Arrondissement,SAINT CHARLES,4094,2696,43.304598,5.389284,1.0,Tea Room,Vegetarian / Vegan Restaurant,Cupcake Shop,Event Service,Electronics Store,Diner,Dessert Shop
3,13001,1er Arrondissement,NOAILLES,4084,2292,43.297432,5.381185,3.0,Hotel,Supermarket,Sporting Goods Shop,Plaza,Café,Nightclub,Dance Studio
4,13001,1er Arrondissement,THIERS,4099,2560,43.297198,5.384647,1.0,Hotel,Seafood Restaurant,French Restaurant,Fountain,Italian Restaurant,Fast Food Restaurant,Metro Station


In [395]:
#Some of the neighborhoods didn't have enough venue places, and then did not get a clustering number as a result. 
#We need to drop these line. And convert Cluster Labels to int.
df_final.dropna(inplace=True)
df_final["Cluster Labels"] = df_final["Cluster Labels"].astype(int)
df_final.head()

,Postal_Code,Borough,Neighborhood,Price_house_m2,Price_appt_m2,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,13001,1er Arrondissement,OPERA,3564,3216,43.293307,5.375022,1,French Restaurant,Pub,Provençal Restaurant,Plaza,Boutique,Coffee Shop,Burger Joint
1,13001,1er Arrondissement,BELSUNCE,4350,2582,43.299124,5.377096,1,Hotel,Bistro,Garden,Indie Movie Theater,Metro Station,Middle Eastern Restaurant,Electronics Store
2,13001,1er Arrondissement,SAINT CHARLES,4094,2696,43.304598,5.389284,1,Tea Room,Vegetarian / Vegan Restaurant,Cupcake Shop,Event Service,Electronics Store,Diner,Dessert Shop
3,13001,1er Arrondissement,NOAILLES,4084,2292,43.297432,5.381185,3,Hotel,Supermarket,Sporting Goods Shop,Plaza,Café,Nightclub,Dance Studio
4,13001,1er Arrondissement,THIERS,4099,2560,43.297198,5.384647,1,Hotel,Seafood Restaurant,French Restaurant,Fountain,Italian Restaurant,Fast Food Restaurant,Metro Station


In [401]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='cartodbpositron')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighborhood'], df_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.3).add_to(map_clusters)
       
map_clusters

In [397]:
df_final.loc[df_final['Cluster Labels'] == 1, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,1er Arrondissement,43.293307,5.375022,1,French Restaurant,Pub,Provençal Restaurant,Plaza,Boutique,Coffee Shop,Burger Joint
1,1er Arrondissement,43.299124,5.377096,1,Hotel,Bistro,Garden,Indie Movie Theater,Metro Station,Middle Eastern Restaurant,Electronics Store
2,1er Arrondissement,43.304598,5.389284,1,Tea Room,Vegetarian / Vegan Restaurant,Cupcake Shop,Event Service,Electronics Store,Diner,Dessert Shop
4,1er Arrondissement,43.297198,5.384647,1,Hotel,Seafood Restaurant,French Restaurant,Fountain,Italian Restaurant,Fast Food Restaurant,Metro Station
5,1er Arrondissement,43.300517,5.384201,1,Plaza,Hotel,Seafood Restaurant,Fountain,Kebab Restaurant,Metro Station,Middle Eastern Restaurant
6,2e Arrondissement,43.297275,5.367222,1,French Restaurant,Plaza,Provençal Restaurant,Tea Room,Hotel,Japanese Restaurant,Café
22,6e Arrondissement,43.283713,5.376230,1,Ice Cream Shop,Café,Vegetarian / Vegan Restaurant,Dance Studio,Event Service,Electronics Store,Diner
24,6e Arrondissement,43.286713,5.383377,1,Brasserie,Sandwich Place,Café,Brewery,Mobile Phone Shop,Metro Station,Restaurant
25,6e Arrondissement,43.290650,5.380547,1,French Restaurant,Plaza,Coffee Shop,Hotel,Bakery,Boutique,Cupcake Shop
26,6e Arrondissement,43.286243,5.387498,1,Hotel,Supermarket,Sandwich Place,Electronics Store,Bar,French Restaurant,Plaza
